# Evaluation of edge detectors
This Jupyter Notebook contains the code to generate ROC auc and other metrics based on confusion matrices. The confusion matrices can be created by comparing (in our case 21 thresholds) thresholded binary edge maps with the ground truths created by the Github repository in the README.md. The confusion matrices should be saved in a .csv file named 
\[algorithm name\]_measures.csv

In [1]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
from itertools import cycle

from scipy.integrate import simps
import scipy
from sklearn.metrics import auc

# np.sqrt has limitations, therefore we use the sqrt from the math library
from math import sqrt

# For the qualitative evaluation
from skimage.filters import farid, roberts, sobel, scharr, prewitt
from edge_detectors import frei, laplacianofgausian, wavelet

In [2]:
def extract_conf_matrix(x):
    ''' Converst a confusion matrix save in string to a dictornary and returns the
        true positives, true negatives, false negative and false postiives
    '''
    x = ast.literal_eval(x) # The dicts are saves as strings, so need to be converted 
    return x['tn'], x['fp'], x['fn'], x['tp']


# The following formulas are taken from https://en.wikipedia.org/wiki/Confusion_matrix
def TPR(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    if TP+FN == 0:
        return 0
    return TP/(TP+FN)
    
def TNR(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    if TN+FP == 0:
        return 0
    return TN/(TN+FP)

def PPV(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    if TP+FP == 0:
        return 0
    return TP/(TP+FP)

def NPV(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    if TN+FN == 0:
        return 0
    return TN/(TN+FN)

def FNR(x):
    return 1 - TPR(x)

def FPR(x):
    '''fall-out or false positive rate (FPR)'''
    return 1 - TNR(x)

def FDR(x):
    return 1 - PPV(x)

def FOR(x):
    return 1 - NPV(x)

def PT(x):
    t = sqrt(TPR(x)*(-TNR(x)+1))+TNR(x)-1
    n = TPR(x)+TNR(x)-1
    return t/n

def TS(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    return TP/(TP+FN+FP)

def ACC(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    return (TP+TN)/(TP+TN+FP+FN)

def BA(x):
    return (TPR(x)+TNR(x))/2

def F1(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    return (2*TP)/(2*TP+FP+FN)

def MCC(x):
    TN, FP, FN, TP = extract_conf_matrix(x)
    t = TP*TN-FP*FN
    n = sqrt(float((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))
    
def FM(x):
    return sqrt(PPV(x)*TPR(x))

def BM(x):
    return TPR(x)+TNR(x)-1

def MK(x):
    return PPV(x)+NPV(x)-1

In [3]:
def show(img, name, vmax=255):
    ''' This helper function makes displaying gray scale images easier'''
    plt.imshow(img, cmap='gray', vmin = 0, vmax = vmax,interpolation='none')
    plt.title(name)
    plt.show()
    
def thresholding(image, threshold):
    '''This helper function makes thresholding easier'''
    # Thresholding
    thresh_image = image.copy()
    thresh_image[image>=threshold]=1
    thresh_image[image<threshold]=0
    return thresh_image

In [4]:
edge_algorithm = input('Please input the name of the algorithm you would like to evalutate (case sensitive)')
df = pd.read_csv(f'{edge_algorithm}_measures.csv')
print(f'{edge_algorithm} confusion matrices')
# Drop columns with no confusion matrixes
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('Image number name')]
df

Please input the name of the algorithm you would like to evalutate (case sensitive)farid


FileNotFoundError: [Errno 2] No such file or directory: 'farid_measures.csv'

In [5]:
def create_measurements(edge_algorithm, measurement):
    ''' Helps return a DataFrame with the values caluated with the input measurement
    '''
    df = pd.read_csv(f'{edge_algorithm}_measures.csv')

    # Drop columns with no confusion matrixes
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.loc[:, ~df.columns.str.contains('Image number name')]
    return df.apply(np.vectorize(measurement))

# Plot precision recall

In [6]:
def plot_pr_algorithms(list_algorithms, f_scores = [0.01, 0.1, 0.2, 0.4, 0.6, 0.8]):
    # setup plot details
    plt.figure(figsize=(7, 8))
    max_x = 0
    max_y = 0
    for edge_detector in list_algorithms:
        df_temp_PPV = create_measurements(edge_detector, PPV)
        df_temp_TPR = create_measurements(edge_detector, TPR)
        
        # Plot for edge detector
        recall_list = [np.mean(df_temp_TPR[column]) for column in df_temp_TPR] # recall_list
        precision_list = [np.mean(df_temp_PPV[column]) for column in df_temp_PPV] # precision_list
        plt.plot(recall_list, precision_list, 
                 label=f'{edge_detector}')
        
        # update max for good fit Iso-F1
        if np.max(recall_list) > max_x:
            max_x = np.max(recall_list)
        if np.max(precision_list) > max_y:
            max_y = np.max(precision_list)
            
        # Show progress
        print(f'{edge_detector} done', end="")
        
    # Plot Iso-F1
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
        plt.annotate(f'f1={f_score}', xy=(0.9, y[45] + 0.02))
    
    plt.xlim(0,max_x)
    plt.ylim(0,max_y)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    plt.show()

## This was my original ROC implementation
def plot_ROC_algorithms(list_algorithms, pretty_names, f_scores = [0.01, 0.1, 0.2, 0.4, 0.6, 0.8]):
    auc_dict = dict()
    
    # setup plot details
    plt.figure(figsize=(7, 8))
    
    for edge_detector, pretty_name in zip(list_algorithms, pretty_names):
        df_temp_FPR = create_measurements(edge_detector, FPR) # x fall-out or false positive rate (FPR)
        df_temp_TPR = create_measurements(edge_detector, TPR) # y sensitivity, recall, hit rate, or true positive rate (TPR)
        
        
        # Plot for edge detector
        FPR_list = [np.mean(df_temp_FPR[column]) for column in df_temp_FPR] # x
        TPR_list = [np.mean(df_temp_TPR[column]) for column in df_temp_TPR] # y
        
        temp_auc_val = auc(FPR_list, TPR_list)
        plt.plot(FPR_list, TPR_list, label=f'{pretty_name}, AUC: {temp_auc_val:.4f}')
        auc_dict[edge_detector] = temp_auc_val
        print(f'{pretty_name} is done, AUC={temp_auc_val}', end=", ")
    

    plt.xlabel('False positives')
    plt.ylabel('True positives')
    plt.legend()
    plt.show()
    return auc_dict

In [7]:
# Add the names of all the edge detector you would like to evaluate
# list_algorithms = ['farid']
# plot_pr_algorithms(list_algorithms) # UNCOMMENT TO USE

# Precision recall curves
To plot the precision recall curves for your edge detection methods you should have the .csv files for every one of them. The names of the edge detectors should be added to the variable ```all_edge_detectors```. An example is shown below:
```python
all_edge_detectors = ['frei', 'prewitt', 'scharr', 'sobel', 'farid', 'roberts', 'gabor_filterbank1', 'GS3O11', 'GS5O11', 'GS5O8', 'k-means', 'canny', 'wavelet', 'LoG_sigma1.4', 'LoG_sigma2.7', 'LoG_sigma2', 'canny_sigma0.1', 'canny_sigma1', 'canny_sigma1.4', 'canny_sigma2.7']
```

In [8]:
# plot_pr_algorithms(all_edge_detectors) # ADD OWN EDGE DETECTOR NAMES AND UNCOMMENT

# ROC curves
For the creation of the ROC curves, the names of the edge detectors in the .csv filenames should be added to ```edge_detectors_without_params```. These names might not containt capitals or some basic explaination. Therefore add the names you would like to display in the legend of the ROC curves to the ```edge_detectors_without_params_pretty``` variable in a list. An example is shown below:
```python
edge_detectors_without_params = ['farid','frei','prewitt','roberts', 'scharr','sobel','wavelet']
edge_detectors_without_params_pretty = ['Farid','Frei-Chen','Prewitt','Roberts','Scharr','Sobel','Wavelet']
auc_dict_no_params = plot_ROC_algorithms(edge_detectors_without_params, edge_detectors_without_params_pretty)
```

In [9]:
# auc_dict_no_params = plot_ROC_algorithms(edge_detectors_without_params, edge_detectors_without_params_pretty) # ADD OWN EDGE DETECTOR NAMES AND UNCOMMENT

# Evaluation of other metrics
To create a DataFrame with metrics, for example F1 score. The ```create_measurements``` definition can be used. This definition will return the DataFrame with the given metric for every threshold and image. An example of the code is shown below:
```python
print(f'F1 values for {edge_algorithm}')
df_F1 = create_measurements(edge_algorithm, F1)
df_F1
```

With this dataframe, averages, maximum values etc. can 